# Exercise 4.2 seaborn
prepared by M.Hauser

Seaborn is a library for statistical visualisation; it tries to 'make a well-defined set of hard things easy to do'.

It has a beautiful [gallery](https://seaborn.pydata.org/examples/index.html) illustrating its capabilities.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr

Let's use a seaborn style:

In [ ]:
sns.set(style="white")

### Load Data

We again use time series of Station Data for Switzerland - Temperature & Precip.

The data is available from [MeteoSwiss](https://www.meteoswiss.admin.ch/home/climate/swiss-climate-in-detail/homogeneous-data-series-since-1864.html?region=Table).

The data has already been [retrieved and postprocessed](../data/prepare_data_MCH.ipynb).

In [ ]:
def load_mch(station, annual=True):
    fN = "../data/MCH_HOM_{}.nc".format(station)
    return xr.open_dataset(fN, drop_variables=["station", "station_long"])


BAS = load_mch("BAS")
DAV = load_mch("DAV")

## Distributions

While it is binned histograms with matplotlib have a dedicated function in matplotlib, it's difficult to plot a Kernel Density Estimate (kde). Seaborn has specialized functions wich allow to do this directly:

In [ ]:
# create random data
d = np.random.randn(100)

# ======================

# Set up the matplotlib figure
f, axs = plt.subplots(2, 2, figsize=(7, 7), sharex=True, sharey=True)
axs = axs.flatten()


# get rid of the left axis
sns.despine(left=True)

ax = axs[0]

# Plot a simple histogram with binsize determined automatically
sns.histplot(d, kde=False, color="b", ax=ax, stat="density")

ax = axs[1]

# Plot a kernel density estimate and rug plot
sns.kdeplot(d, color="r", ax=ax)
sns.rugplot(d, color="r", ax=ax)

ax = axs[2]

# Plot a filled kernel density estimate
# sns.distplot(d, hist=False, color="g", kde_kws={"shade": True}, ax=ax)
sns.kdeplot(d, color="g", fill=True, ax=ax)


ax = axs[3]

# Plot a historgram and kernel density estimate
sns.kdeplot(d, color="m", ax=ax)
sns.histplot(d, kde=False, color="m", ax=ax, stat="density")


plt.setp(axs, yticks=[])
plt.tight_layout()

### Exercise

 * Add a `kdeplot` of `BAS.Temperature` and `DAV.Temperature`
 * can you add a legend?

In [ ]:
f, ax = plt.subplots()

# code here

### Solution

In [ ]:
f, ax = plt.subplots()

sns.kdeplot(BAS.Temperature, fill=True, ax=ax, label="Basel")
sns.kdeplot(DAV.Temperature, fill=True, ax=ax, label="Davos")

plt.legend();

## Joint plot

`jointplot` allows you to see the distribution of two individual data sets as well as their joint distribution

In [ ]:
sns.jointplot(x=BAS.Temperature, y=BAS.Precipitation, kind="kde");

### Exercise
 * is there a correlation between precipitation and temperature in Davos (`DAV`)
 * choose another `kind`

In [ ]:
# uncomment to get the docstring
# sns.jointplot?

### Solution

In [ ]:
sns.jointplot(x=DAV.Temperature, y=DAV.Precipitation, kind="hex");

## Pandas

Seaborn works especially well with pandas dataframes. We can illustrate this with an example from the [seaborn gallery](https://seaborn.pydata.org/examples/factorplot_bars.html).

The example data is a passenger list from the titanic:

In [ ]:
# Load the example Titanic dataset
titanic = sns.load_dataset("titanic")

titanic.head()

Then we can use a `catplot`/`factorplot` to illustrate the survival probability depending on the class the passenger traveled and if it was a male or female passenger:

In [ ]:
with sns.axes_style("whitegrid"):
    # Draw a nested barplot to show survival for class and sex
    g = sns.catplot(
        x="class",
        y="survived",
        hue="sex",
        data=titanic,
        height=6,
        kind="bar",
        palette="muted",
    )

    g.despine(left=True)
    g.set_ylabels("survival probability")

Pandas DataFrame don't work very well with lat/ lon data (that's what xarray and the like are for), so let's use an example with a time series. First we need to convert `BAS` from an xarray Dataset to a pandas DataFrame. 

In [ ]:
import pandas as pd

In [ ]:
def to_dataframe(data):
    # STEP 1
    # calculate monthly temperature and precipitation anomalies
    d = data.groupby("time.month") - data.groupby("time.month").mean("time")

    # STEP 2
    # convert to a dataframe
    d = d.to_dataframe()[["Temperature", "Precipitation"]]

    # STEP 3
    # create a new categorical variable 'month
    d["month"] = d.index.month.values
    d["month"] = d["month"].astype("category")

    # STEP 4
    # create wet and dry category depending if it rained more than on average
    bins = [-np.inf, 0, np.inf]
    d["prec_cat"] = pd.cut(d["Precipitation"], bins, labels=["dry", "wet"])

    return d


BAS_df = to_dataframe(BAS)
DAV_df = to_dataframe(DAV)
BAS_df.head()

### Exercise

 * Create a `catplot` showing monthly temperature anomalies as a function of the `month` and precipitation category

In [ ]:
# code here

# sns.catplot(...)

### Solution

In [ ]:
g = sns.catplot(
    x="month",
    y="Temperature",
    hue="prec_cat",
    data=BAS_df,
    height=6,
    kind="bar",
    palette="BrBG",
)